# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
    !tar xf spark-3.2.0-bin-hadoop3.2.tgz
    !mv spark-3.2.0-bin-hadoop3.2 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ColabSparkApp") \
    .getOrCreate()

# Verify Spark session
spark

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [5]:
# df = spark.read.csv("/content/my_file.csv", header=True, inferSchema=True)
df = spark.read.csv("netflix-rotten-tomatoes-metacritic-imdb.csv", header=True,inferSchema=True)
df.show()
df.printSchema()
df.describe().show()
print(df.count())
print(df.columns)

+--------------------+--------------------+--------------------+--------------------+---------------+----------------+--------------------+------------+--------------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|               Title|               Genre|                Tags|           Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|            Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For|   Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|           

## What is the maximum and average of the overall hidden gem score?

In [6]:
from pyspark.sql import functions as F
HIDDEN_GEM_SCORE=("Hidden Gem Score")
average_hidden_gem_score = df.select(F.avg(HIDDEN_GEM_SCORE)).collect()[0][0]
print(f'average_hidden_gem_score: {average_hidden_gem_score}')
max_hidden_gem_score = df.select(F.max(HIDDEN_GEM_SCORE)).collect()[0][0]
print(f'max_hidden_gem_score: {max_hidden_gem_score}')

average_hidden_gem_score: 5.937551386501234
max_hidden_gem_score: 9.8


## How many movies that are available in Korea?

In [7]:
number_of_movies_available_in_korea = df.filter(df["Languages"].rlike('Korea')).count()
print(number_of_movies_available_in_korea)

735


## Which director has the highest average hidden gem score?

In [8]:
DIRECTOR='Director'
director,score=df.groupBy(DIRECTOR).agg(
    F.avg(HIDDEN_GEM_SCORE).alias("Average_Hidden_Gem_Score")
).orderBy(F.col("Average_Hidden_Gem_Score").desc()).first()
print(f'director: {director}, score: {score}')

director: Dorin Marcu, score: 9.8


## How many genres are there in the dataset?

In [17]:
GENRE = 'Genre'
genres = [genre[0] for genre in (
    df.withColumn(GENRE, F.split(df[GENRE],","))
    .withColumn(GENRE, F.explode(F.col(GENRE)))
    .withColumn(GENRE, F.trim(F.col(GENRE)))
    ).select(GENRE).collect()]

print(f'number of distinct genres: {len(set(genres))}')


number of distinct genres: 28
